# Classification with Pyspark
>  Now that you are familiar with getting data into Spark, you'll move onto building two types of classification model: Decision Trees and Logistic Regression. You'll also find out about a few approaches to data preparation.

- toc: true 
- badges: true
- comments: true
- author: Lucas Nunes
- categories: [Datacamp]
- image: images/datacamp/___

> Note: This is a summary of the course's chapter 2 exercises "Machine Learning with PySpark" at datacamp. <br>[Github repo](https://github.com/lnunesAI/Datacamp/) / [Course link](https://www.datacamp.com/tracks/machine-learning-scientist-with-python)

In [ ]:
%%capture
!pip install pyspark
import pyspark
import pandas as pd
import numpy as np
from pyspark.sql import SparkSession

!wget https://github.com/lnunesAI/Datacamp/raw/main/2-machine-learning-scientist-with-python/22-machine-learning-with-pyspark/datasets/flights.csv
spark = SparkSession.builder.master('local[*]').appName('flights').getOrCreate()

In [ ]:
flights = spark.read.csv('flights.csv', sep=',', header=True, inferSchema=True, nullValue='NA')

## Data Preparation

### Removing columns and rows

<div class=""><p>You previously loaded airline flight data from a CSV file. You're going to develop a model which will predict whether or not a given flight will be delayed.</p>
<p>In this exercise you need to trim those data down by:</p>
<ol>
<li>removing an uninformative column and</li>
<li>removing rows which do not have information about whether or not a flight was delayed.</li>
</ol>
<p>The data are available as <code>flights</code>.</p></div>

Instructions
<ul>
<li>Remove the <code>flight</code> column.</li>
<li>Find out how many records have missing values in the <code>delay</code> column.</li>
<li>Remove records with missing values in the <code>delay</code> column.</li>
<li>Remove records with missing values in any column and get the number of remaining rows.</li>
</ul>

In [ ]:
# Remove the 'flight' column
flights_drop_column = flights.drop('flight')

# Number of records with missing 'delay' values
flights_drop_column.filter('delay IS NULL').count()

# Remove records with missing 'delay' values
flights_valid_delay = flights_drop_column.filter('delay IS NOT NULL')

# Remove records with missing values in any column and get the number of remaining rows
flights_none_missing = flights_valid_delay.dropna()
print(flights_none_missing.count())

47022


**You've discarded the columns and rows which will certainly not contribute to a model.**

### Column manipulation

<div class=""><p>The Federal Aviation Administration (FAA) considers a flight to be "delayed" when it arrives 15 minutes or more after its scheduled time.</p>
<p>The next step of preparing the flight data has two parts:</p>
<ol>
<li>convert the units of distance, replacing the <code>mile</code> column with a <code>km</code>column; and</li>
<li>create a Boolean column indicating whether or not a flight was delayed.</li>
</ol></div>

In [ ]:
flights = flights_none_missing

Instructions
<ul>
<li>Import a function which will allow you to round a number to a specific number of decimal places.</li>
<li>Derive a new <code>km</code> column from the <code>mile</code> column, rounding to zero decimal places. One mile is 1.60934 km.</li>
<li>Remove the <code>mile</code> column.</li>
<li>Create a <code>label</code> column with a value of 1 indicating the delay was 15 minutes or more and 0 otherwise.</li>
</ul>

In [ ]:
# Import the required function
from pyspark.sql.functions import round

# Convert 'mile' to 'km' and drop 'mile' column
flights_km = flights.withColumn('km', round(flights.mile * 1.60934, 0)) \
                    .drop('mile')

# Create 'label' column indicating whether flight delayed (1) or not (0)
flights_km = flights_km.withColumn('label', (flights.delay >= 15).cast('integer'))

# Check first five records
flights_km.show(5)

+---+---+---+-------+---+------+--------+-----+------+-----+
|mon|dom|dow|carrier|org|depart|duration|delay|    km|label|
+---+---+---+-------+---+------+--------+-----+------+-----+
|  0| 22|  2|     UA|ORD| 16.33|      82|   30| 509.0|    1|
|  2| 20|  4|     UA|SFO|  6.17|      82|   -8| 542.0|    0|
|  9| 13|  1|     AA|ORD| 10.33|     195|   -5|1989.0|    0|
|  5|  2|  1|     UA|SFO|  7.98|     102|    2| 885.0|    0|
|  7|  2|  6|     AA|ORD| 10.83|     135|   54|1180.0|    1|
+---+---+---+-------+---+------+--------+-----+------+-----+
only showing top 5 rows



**Fifteen minutes seems like quite a wide margin, but who are you to argue with the FAA?**

### Categorical columns

<p>In the flights data there are two columns, <code>carrier</code> and <code>org</code>, which hold categorical data. You need to transform those columns into indexed numerical values.</p>

In [ ]:
flights = flights_km

Instructions
<ul>
<li>Import the appropriate class and create an indexer object to transform the <code>carrier</code> column from a string to an numeric index.</li>
<li>Prepare the indexer object on the flight data.</li>
<li>Use the prepared indexer to create the numeric index column.</li>
<li>Repeat the process for the <code>org</code> column.</li>
</ul>

In [ ]:
from pyspark.ml.feature import StringIndexer

# Create an indexer
indexer = StringIndexer(inputCol='carrier', outputCol='carrier_idx')

# Indexer identifies categories in the data
indexer_model = indexer.fit(flights)

# Indexer creates a new column with numeric index values
flights_indexed = indexer_model.transform(flights)

# Repeat the process for the other categorical feature
flights_indexed = StringIndexer(inputCol='org', outputCol='org_idx').fit(flights_indexed).transform(flights_indexed)

**Our Machine Learning model needs numbers not strings, so these transformations are vital!**

### Assembling columns

<div class=""><p>The final stage of data preparation is to consolidate all of the predictor columns into a single column.</p>
<p>An <em>updated</em> version of the <code>flights</code> data, which takes into account all of the changes from the previous few exercises, has the following predictor columns:</p>
<ul>
<li><code>mon</code>, <code>dom</code> and <code>dow</code></li>
<li><code>carrier_idx</code> (indexed value from <code>carrier</code>)</li>
<li><code>org_idx</code> (indexed value from <code>org</code>)</li>
<li><code>km</code></li>
<li><code>depart</code></li>
<li><code>duration</code></li>
</ul></div>

In [ ]:
flights = flights_indexed

Instructions
<ul>
<li>Import the class which will assemble the predictors.</li>
<li>Create an assembler object that will allow you to merge the predictors columns into a single column.</li>
<li>Use the assembler to generate a new consolidated column.</li>
</ul>

In [ ]:
# Import the necessary class
from pyspark.ml.feature import VectorAssembler

# Create an assembler object
assembler = VectorAssembler(inputCols=['mon', 'dom', 'dow', 
'carrier_idx', 'org_idx', 'km', 
'depart', 'duration'], outputCol='features')

# Consolidate predictor columns
flights_assembled = assembler.transform(flights)

# Check the resulting column
flights_assembled.select('features', 'delay').show(5, truncate=False)

+-----------------------------------------+-----+
|features                                 |delay|
+-----------------------------------------+-----+
|[0.0,22.0,2.0,0.0,0.0,509.0,16.33,82.0]  |30   |
|[2.0,20.0,4.0,0.0,1.0,542.0,6.17,82.0]   |-8   |
|[9.0,13.0,1.0,1.0,0.0,1989.0,10.33,195.0]|-5   |
|[5.0,2.0,1.0,0.0,1.0,885.0,7.98,102.0]   |2    |
|[7.0,2.0,6.0,1.0,0.0,1180.0,10.83,135.0] |54   |
+-----------------------------------------+-----+
only showing top 5 rows



**The data are now ready for building our first Machine Learning model. You've worked hard to get this sorted: well done!**

## Decision Tree

### Train/test split

<div class=""><p>To objectively assess a Machine Learning model you need to be able to test it on an independent set of data. You can't use the same data that you used to train the model: of course the model will perform (relatively) well on those data!</p>
<p>You will split the data into two components:</p>
<ul>
<li>training data (used to train the model) and</li>
<li>testing data (used to test the model).</li>
</ul></div>

In [ ]:
flights = flights_assembled

Instructions
<ul>
<li>Randomly split the <code>flights</code> data into two sets with 80:20 proportions. For repeatability set a random number seed of 17 for the split.</li>
<li>Check that the training data has roughly 80% of the records from the original data.</li>
</ul>

In [ ]:
# Split into training and testing sets in a 80:20 ratio
flights_train, flights_test = flights.randomSplit([.8, .2], seed=17)

# Check that training set has around 80% of records
training_ratio = flights_train.count() / flights.count()
print(training_ratio)

0.796967376972481


**The ratio looks as expected. You're ready to train and test a Decision Tree model!**

### Build a Decision Tree

<div class=""><p>Now that you've split the flights data into training and testing sets, you can use the training set to fit a Decision Tree model.</p>
<p>The data are available as <code>flights_train</code> and <code>flights_test</code>.</p>
<p>NOTE: It will take a few seconds for the model to train… please be patient!</p></div>

Instructions
<ul>
<li>Import the class for creating a Decision Tree classifier.</li>
<li>Create a classifier object and fit it to the training data.</li>
<li>Make predictions for the testing data and take a look at the predictions.</li>
</ul>

In [ ]:
# Import the Decision Tree Classifier class
from pyspark.ml.classification import DecisionTreeClassifier

# Create a classifier object and fit to the training data
tree = DecisionTreeClassifier()
tree_model = tree.fit(flights_train)

# Create predictions for the testing data and take a look at the predictions
prediction = tree_model.transform(flights_test)
prediction.select('label', 'prediction', 'probability').show(5, False)

+-----+----------+----------------------------------------+
|label|prediction|probability                             |
+-----+----------+----------------------------------------+
|1    |0.0       |[0.5331923890063425,0.46680761099365753]|
|0    |1.0       |[0.353643525356967,0.646356474643033]   |
|0    |0.0       |[0.6903781355297641,0.3096218644702359] |
|1    |1.0       |[0.353643525356967,0.646356474643033]   |
|1    |1.0       |[0.353643525356967,0.646356474643033]   |
+-----+----------+----------------------------------------+
only showing top 5 rows



**You've built your first Machine Learning model with PySpark.**

### Evaluate the Decision Tree

<div class=""><p>You can assess the quality of your model by evaluating how well it performs on the testing data. Because the model was not trained on these data, this represents an objective assessment of the model.</p>
<p>A <em>confusion matrix</em> gives a useful breakdown of predictions versus known values. It has four cells which represent the counts of:</p>
<ul>
<li><em>True Negatives</em> (TN) — model predicts negative outcome &amp; known outcome is negative</li>
<li><em>True Positives</em> (TP) — model predicts positive outcome &amp; known outcome is positive</li>
<li><em>False Negatives</em> (FN) —  model predicts negative outcome but known outcome is positive</li>
<li><em>False Positives</em> (FP) — model predicts positive outcome but known outcome is negative.</li>
</ul></div>

Instructions
<ul>
<li>Create a confusion matrix by counting the combinations of <code>label</code> and <code>prediction</code>. Display the result.</li>
<li>Count the number of True Negatives, True Positives, False Negatives and False Positives.</li>
<li>Calculate the accuracy.</li>
</ul>

In [ ]:
# Create a confusion matrix
prediction.groupBy('label', 'prediction').count().show()

# Calculate the elements of the confusion matrix
TN = prediction.filter('prediction = 0 AND label = prediction').count()
TP = prediction.filter('prediction = 1 AND label = prediction').count()
FN = prediction.filter('prediction = 0 AND label = 1').count()
FP = prediction.filter('prediction = 1 AND label = 0').count()

# Accuracy measures the proportion of correct predictions
accuracy = (TN+TP)/(TN+TP+FN+FP)
print(accuracy)

+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|    1|       0.0| 1723|
|    0|       0.0| 2818|
|    1|       1.0| 3187|
|    0|       1.0| 1819|
+-----+----------+-----+

0.6289934010683984


**The accuracy is decent but there are a lot of false predictions. We can make this model better!**

## Logistic Regression

### Build a Logistic Regression model

<div class=""><p>You've already built a Decision Tree model using the flights data. Now you're going to create a Logistic Regression model on the same data.</p>
<p>The objective is to predict whether a flight is likely to be delayed by at least 15 minutes (label <code>1</code>) or not (label <code>0</code>).</p>
<p>Although you have a variety of predictors at your disposal, you'll only use the <code>mon</code>, <code>depart</code> and <code>duration</code> columns for the moment. These are numerical features which can immediately be used for a Logistic Regression model. You'll need to do a little more work before you can include categorical features. Stay tuned!</p>
<p>The data have been split into training and testing sets and are available as <code>flights_train</code> and <code>flights_test</code>.</p></div>

Instructions
<ul>
<li>Import the class for creating a Logistic Regression classifier.</li>
<li>Create a classifier object and train it on the training data.</li>
<li>Make predictions for the testing data and create a confusion matrix.</li>
</ul>

In [ ]:
# Import the logistic regression class
from pyspark.ml.classification import LogisticRegression

# Create a classifier object and train on training data
logistic = LogisticRegression().fit(flights_train)

# Create predictions for the testing data and show confusion matrix
prediction = logistic.transform(flights_test)
prediction.groupBy('label', 'prediction').count().show()

+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|    1|       0.0| 1671|
|    0|       0.0| 2522|
|    1|       1.0| 3239|
|    0|       1.0| 2115|
+-----+----------+-----+



### Evaluate the Logistic Regression model

<div class=""><p>Accuracy is generally not a very reliable metric because it can be biased by the most common target class.</p>
<p>There are two other useful metrics:</p>
<ul>
<li><em>precision</em> and</li>
<li><em>recall</em>.</li>
</ul>
<p>Check the slides for this lesson to get the relevant expressions.</p>
<p>Precision is the proportion of positive predictions which are correct. For all flights which are predicted to be delayed, what proportion is actually delayed?</p>
<p>Recall is the proportion of positives outcomes which are correctly predicted. For all delayed flights, what proportion is correctly predicted by the model?</p>
<p>The precision and recall are generally formulated in terms of the positive target class. But it's also possible to calculate <em>weighted</em> versions of these metrics which look at both target classes.</p>
<p>The components of the confusion matrix are available as <code>TN</code>, <code>TP</code>, <code>FN</code> and <code>FP</code>, as well as the object <code>prediction</code>.</p></div>

Instructions
<ul>
<li>Find the precision and recall.</li>
<li>Create a multi-class evaluator and evaluate weighted precision.</li>
<li>Create a binary evaluator and evaluate AUC using the <code>"areaUnderROC"</code> metric.</li>
</ul>

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator, BinaryClassificationEvaluator

# Calculate precision and recall
precision = TP / (TP + FP)
recall = TP / (TP + FN)
print('precision = {:.2f}\nrecall    = {:.2f}'.format(precision, recall))

# Find weighted precision
multi_evaluator = MulticlassClassificationEvaluator()
weighted_precision = multi_evaluator.evaluate(prediction, {multi_evaluator.metricName: "weightedPrecision"})

# Find AUC
binary_evaluator = BinaryClassificationEvaluator()
auc = binary_evaluator.evaluate(prediction, {binary_evaluator.metricName:'areaUnderROC'})

precision = 0.64
recall    = 0.65


In [ ]:
weighted_precision, auc

(0.6032733445738772, 0.6397733716273997)

**The weighted precision indicates what proportion of predictions (positive and negative) are correct.**

## Turning Text into Tables

### Punctuation, numbers and tokens

<div class=""><p>At the end of the previous chapter you loaded a dataset of SMS messages which had been labeled as either "spam" (label <code>1</code>) or "ham" (label <code>0</code>). You're now going to use those data to build a classifier model.</p>
<p>But first you'll need to prepare the SMS messages as follows:</p>
<ul>
<li>remove punctuation and numbers</li>
<li>tokenize (split into individual words)</li>
<li>remove stop words</li>
<li>apply the hashing trick</li>
<li>convert to TF-IDF representation.</li>
</ul>
<p>In this exercise you'll remove punctuation and numbers, then tokenize the messages.</p>
<p>The SMS data are available as <code>sms</code>.</p></div>

In [ ]:
%%capture
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
!wget https://github.com/lnunesAI/Datacamp/raw/main/2-machine-learning-scientist-with-python/22-machine-learning-with-pyspark/datasets/sms.csv
schema = StructType([
    StructField("id", IntegerType()),
    StructField("text", StringType()),
    StructField("label", IntegerType())
])
sms = spark.read.csv('sms.csv', sep=';', header=False, schema=schema)

Instructions
<ul>
<li>Import the function to replace regular expressions and the feature to tokenize.</li>
<li>Replace all punctuation characters from the <code>text</code> column with a space. Do the same for all numbers in the <code>text</code> column.</li>
<li>Split the <code>text</code> column into tokens. Name the output column <code>words</code>.</li>
</ul>

In [ ]:
# Import the necessary functions
from pyspark.sql.functions import regexp_replace
from pyspark.ml.feature import Tokenizer

# Remove punctuation (REGEX provided) and numbers
wrangled = sms.withColumn('text', regexp_replace(sms.text, '[_():;,.!?\\-]', ' '))
wrangled = wrangled.withColumn('text', regexp_replace(wrangled.text, '[0-9]', ' '))

# Merge multiple spaces
wrangled = wrangled.withColumn('text', regexp_replace(wrangled.text, ' +', ' '))

# Split the text into words
wrangled = Tokenizer(inputCol='text', outputCol='words').transform(wrangled)

wrangled.show(4, truncate=False)

+---+----------------------------------+-----+------------------------------------------+
|id |text                              |label|words                                     |
+---+----------------------------------+-----+------------------------------------------+
|1  |Sorry I'll call later in meeting  |0    |[sorry, i'll, call, later, in, meeting]   |
|2  |Dont worry I guess he's busy      |0    |[dont, worry, i, guess, he's, busy]       |
|3  |Call FREEPHONE now                |1    |[call, freephone, now]                    |
|4  |Win a cash prize or a prize worth |1    |[win, a, cash, prize, or, a, prize, worth]|
+---+----------------------------------+-----+------------------------------------------+
only showing top 4 rows



### Stop words and hashing

<div class=""><p>The next steps will be to remove stop words and then apply the hashing trick, converting the results into a TF-IDF.</p>
<p>A quick reminder about these concepts:</p>
<ul>
<li>The hashing trick provides a fast and space-efficient way to map a very large (possibly infinite) set of items (in this case, all words contained in the SMS messages) onto a smaller, finite number of values.</li>
<li>The TF-IDF matrix reflects how important a word is to each document. It takes into account both the frequency of the word within each document but also the frequency of the word across all of the documents in the collection.</li>
</ul>
<p>The tokenized SMS data are stored in <code>sms</code> in a column named <code>words</code>. You've cleaned up the handling of spaces in the data so that the tokenized text is neater.</p></div>

In [ ]:
sms = wrangled.select('id', 'words', 'label')

Instructions
<ul>
<li>Import the <code>StopWordsRemover</code>, <code>HashingTF</code> and <code>IDF</code> classes.</li>
<li>Create a <code>StopWordsRemover</code> object (input column <code>words</code>, output column <code>terms</code>). Apply to <code>sms</code>.</li>
<li>Create a <code>HashingTF</code> object (input results from previous step, output column <code>hash</code>). Apply to <code>wrangled</code>.</li>
<li>Create an <code>IDF</code> object (input results from previous step, output column <code>features</code>). Apply to <code>wrangled</code>.</li>
</ul>

In [ ]:
from pyspark.ml.feature import StopWordsRemover, HashingTF, IDF

# Remove stop words.
wrangled = StopWordsRemover(inputCol='words', outputCol='terms').transform(sms)

# Apply the hashing trick
wrangled = HashingTF(inputCol='terms', outputCol='hash', numFeatures=1024)\
      .transform(wrangled)

# Convert hashed symbols to TF-IDF
tf_idf = IDF(inputCol='hash', outputCol='features')\
      .fit(wrangled).transform(wrangled)
      
tf_idf.select('terms', 'features').show(4, truncate=False)

+--------------------------------+----------------------------------------------------------------------------------------------------+
|terms                           |features                                                                                            |
+--------------------------------+----------------------------------------------------------------------------------------------------+
|[sorry, call, later, meeting]   |(1024,[138,384,577,996],[2.273418200008753,3.6288353225642043,3.5890949939146903,4.104259019279279])|
|[dont, worry, guess, busy]      |(1024,[215,233,276,329],[3.9913186080986836,3.3790235241678332,4.734227298217693,4.58299632849377]) |
|[call, freephone]               |(1024,[133,138],[5.367951058306837,2.273418200008753])                                              |
|[win, cash, prize, prize, worth]|(1024,[31,47,62,389],[3.6632029660684124,4.754846585420428,4.072170704727778,7.064594791043114])    |
+--------------------------------+--------------

### Training a spam classifier

<div class=""><p>The SMS data have now been prepared for building a classifier. Specifically, this is what you have done:</p>
<ul>
<li>removed numbers and punctuation</li>
<li>split the messages into words (or "tokens")</li>
<li>removed stop words</li>
<li>applied the hashing trick and</li>
<li>converted to a TF-IDF representation.</li>
</ul>
<p>Next you'll need to split the TF-IDF data into training and testing sets. Then you'll use the training data to fit a Logistic Regression model and finally evaluate the performance of that model on the testing data.</p>
<p>The data are stored in <code>sms</code> and <code>LogisticRegression</code> has been imported for you.</p></div>

In [ ]:
sms = tf_idf.select('label', 'features')

Instructions
<ul>
<li>Split the data into training and testing sets in a 4:1 ratio. Set the random number seed to 13 to ensure repeatability.</li>
<li>Create a <code>LogisticRegression</code> object and fit it to the training data.</li>
<li>Generate predictions on the testing data.</li>
<li>Use the predictions to form a confusion matrix.</li>
</ul>

In [ ]:
# Split the data into training and testing sets
sms_train, sms_test = sms.randomSplit([0.8, 0.2], seed=13)

# Fit a Logistic Regression model to the training data
logistic = LogisticRegression(regParam=0.2).fit(sms_train)

# Make predictions on the testing data
prediction = logistic.transform(sms_test)

# Create a confusion matrix, comparing predictions to known labels
prediction.groupBy('label', 'prediction').count().show()

+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|    1|       0.0|   39|
|    0|       0.0|  932|
|    1|       1.0|  121|
|    0|       1.0|    4|
+-----+----------+-----+

